In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.metrics import mean_squared_error

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
def oversample(df):
    classes = df.labels.value_counts().to_dict()
    most = max(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df['labels'] == key]) 
    classes_sample = []
    for i in range(1,len(classes_list)):
        classes_sample.append(classes_list[i].sample(most, replace=True))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe,classes_list[0]], axis=0)
    final_df = final_df.reset_index(drop=True)
    return final_df

#**SBIC**

In [ ]:
#SBIC Strict Disagreement
SBIC_Disagreement = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/SBIC_Disagreement.csv')

In [ ]:
SBIC_Disagreement.columns

Index(['text', 'WorkerId', 'annotatorGender', 'annotatorRace', 'annotatorAge',
       'annotatorPolitics', 'offensiveYN', 'major_vote', 'annotators_count',
       'text_label_count', 'person_disagreement_rate', 'disagreement_rate',
       'group_disagreement', 'person_disagreement',
       'normalized_disagreement_rate', 'normalized_person_disagreement_rate'],
      dtype='object')

In [ ]:
#input: demographic + text
#output: personal disagreement 
SBIC_Person_Disagreement = SBIC_Disagreement[['person_disagreement']]
SBIC_Person_Disagreement['demographics_text'] = ["The annotator is a " + str(int(SBIC_Disagreement['annotatorAge'].iloc[i])) + ' years old ' + str(SBIC_Disagreement['annotatorPolitics'].iloc[i]) + ' ' + str(SBIC_Disagreement['annotatorRace'].iloc[i]) + ' ' + str(SBIC_Disagreement['annotatorGender'].iloc[i]) + ". " + str(SBIC_Disagreement['text'].iloc[i]) for i in range(len(SBIC_Disagreement))]
SBIC_Person_Disagreement = SBIC_Person_Disagreement.rename(columns={'demographics_text':'text','person_disagreement': 'labels'})

train_SBIC_Person_Disagreement, validate_SBIC_Person_Disagreement, test_SBIC_Person_Disagreement = np.split(SBIC_Person_Disagreement.sample(frac=1, random_state=42),[int(.6*len(SBIC_Person_Disagreement)), int(.8*len(SBIC_Person_Disagreement))])
train_SBIC_Person_Disagreement_oversampled = oversample(train_SBIC_Person_Disagreement)

# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_SBIC_Person_Disagreement_oversampled, eval_df = validate_SBIC_Person_Disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_SBIC_Person_Disagreement)
mean_squared_error(test_SBIC_Person_Disagreement['labels'], model_outputs)


In [ ]:
print(len(test_SBIC_Person_Disagreement['labels']),len(model_outputs))

17984 17984


In [ ]:
test_SBIC_Person_Disagreement['labels'].value_counts()

In [ ]:
SBIC_Group_Disagreement_reg = SBIC_Disagreement[['text', 'normalized_disagreement_rate']]
SBIC_Group_Disagreement_reg = SBIC_Group_Disagreement_reg.rename(columns={ 'normalized_disagreement_rate': 'labels'}).drop_duplicates()
SBIC_Group_Disagreement_reg['labels'].value_counts()

0.0    18642
0.5     9679
1.0     1652
Name: labels, dtype: int64

In [ ]:
#change demographic template to age:25, race: white, text: ...
SBIC_Disagreement['demographics_text_newtemp'] = ["age: " + str(int(SBIC_Disagreement['annotatorAge'].iloc[i])) + ', politics: ' + str(SBIC_Disagreement['annotatorPolitics'].iloc[i]) + ', race: ' + str(SBIC_Disagreement['annotatorRace'].iloc[i]) + ', gender ' + str(SBIC_Disagreement['annotatorGender'].iloc[i]) + ". text: " + str(SBIC_Disagreement['text'].iloc[i]) for i in range(len(SBIC_Disagreement))]
SBIC_demo_text_newtemp_Group_Disagreement_reg = SBIC_Disagreement[['demographics_text_newtemp', 'normalized_disagreement_rate']]
SBIC_demo_text_newtemp_Group_Disagreement_reg = SBIC_demo_text_newtemp_Group_Disagreement_reg.rename(columns={'demographics_text_newtemp':'text','normalized_disagreement_rate': 'labels'}).drop_duplicates()

train_SBIC_demo_text_newtemp_Group_reg, validate_SBIC_demo_text_newtemp_Group_reg, test_SBIC_demo_text_newtemp_Group_reg = np.split(SBIC_demo_text_newtemp_Group_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(SBIC_demo_text_newtemp_Group_Disagreement_reg)), int(.8*len(SBIC_demo_text_newtemp_Group_Disagreement_reg))])
train_SBIC_demo_text_newtemp_Group_reg_oversampled = oversample(train_SBIC_demo_text_newtemp_Group_reg)

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_SBIC_demo_text_newtemp_Group_reg_oversampled, eval_df = validate_SBIC_demo_text_newtemp_Group_reg)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_SBIC_demo_text_newtemp_Group_reg)
mean_squared_error(test_SBIC_demo_text_newtemp_Group_reg['labels'], model_outputs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/100209 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/12527 [00:00<?, ?it/s]

...

  0%|          | 0/17906 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/2239 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.045854265285980716}


0.04584162916955823

In [ ]:
test_SBIC_demo_text_newtemp_Group_reg['labels'].value_counts()

0.0    11200
0.5     5789
1.0      917
Name: labels, dtype: int64

In [ ]:
SBIC_Disagreement['demographics_text'] = ["The annotator is a " + str(int(SBIC_Disagreement['annotatorAge'].iloc[i])) + ' years old ' + str(SBIC_Disagreement['annotatorPolitics'].iloc[i]) + ' ' + str(SBIC_Disagreement['annotatorRace'].iloc[i]) + ' ' + str(SBIC_Disagreement['annotatorGender'].iloc[i]) + ". " + str(SBIC_Disagreement['text'].iloc[i]) for i in range(len(SBIC_Disagreement))]
SBIC_demo_text_Group_Disagreement_reg = SBIC_Disagreement[['demographics_text', 'normalized_disagreement_rate']]
SBIC_demo_text_Group_Disagreement_reg = SBIC_demo_text_Group_Disagreement_reg.rename(columns={'demographics_text':'text','normalized_disagreement_rate': 'labels'}).drop_duplicates()

train_SBIC_demo_text_Group_reg, validate_SBIC_demo_text_Group_reg, test_SBIC_demo_text_Group_reg = np.split(SBIC_demo_text_Group_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(SBIC_demo_text_Group_Disagreement_reg)), int(.8*len(SBIC_demo_text_Group_Disagreement_reg))])
train_SBIC_demo_text_Group_reg_oversampled = oversample(train_SBIC_demo_text_Group_reg)

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_SBIC_demo_text_Group_reg_oversampled, eval_df = validate_SBIC_demo_text_Group_reg)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_SBIC_demo_text_Group_reg)
mean_squared_error(test_SBIC_demo_text_Group_reg['labels'], model_outputs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/100209 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/12527 [00:00<?, ?it/s]

...

  0%|          | 0/17906 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/2239 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.047592540389685356}


0.04760775441501572

In [ ]:
test_SBIC_demo_text_Group_reg['labels'].value_counts()

0.0    11200
0.5     5789
1.0      917
Name: labels, dtype: int64

In [ ]:
SBIC_Disagreement['demographics'] = [" a " + str(int(SBIC_Disagreement['annotatorAge'].iloc[i])) + ' years old ' + str(SBIC_Disagreement['annotatorPolitics'].iloc[i]) + ' ' + str(SBIC_Disagreement['annotatorRace'].iloc[i]) + ' ' + str(SBIC_Disagreement['annotatorGender'].iloc[i])  for i in range(len(SBIC_Disagreement))]
SBIC_Disagreement['group_demographic'] = SBIC_Disagreement.groupby(['text'])['demographics'].transform(lambda x:'The annotators are' + ', and'.join(x) + '. ')

In [ ]:
# common demographic + text
SBIC_Disagreement['group_common_demographic_text'] = 'age: 36, politics: liberal, race: white, gender: women. text: ' + SBIC_Disagreement['text']

In [ ]:
SBIC_Disagreement['group_demographic_text'] = SBIC_Disagreement['group_demographic'] + SBIC_Disagreement['text']

In [ ]:
SBIC_Disagreement['group_demographic_text'].iloc[0]

"The annotators are a 45 years old liberal white woman, and a 35 years old mod-liberal white man, and a 23 years old liberal asian man. i'm convinced that some of y'all bitches get pregnant purposely because birth control amp plan b pills are effective #128533#128056#9749#65039"

In [ ]:
#input: common demographic + text
SBIC_Group_common_demographic_text_Disagreement_reg = SBIC_Disagreement[['group_common_demographic_text', 'normalized_disagreement_rate']]
SBIC_Group_common_demographic_text_Disagreement_reg = SBIC_Group_common_demographic_text_Disagreement_reg.rename(columns={'group_common_demographic_text':'text' ,'normalized_disagreement_rate': 'labels'}).drop_duplicates()
SBIC_Group_common_demographic_text_Disagreement_reg['labels'].value_counts()

train_Group_common_demographic_text_reg, validate_Group_common_demographic_text_reg, test_Group_common_demographic_text_reg = np.split(SBIC_Group_common_demographic_text_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(SBIC_Group_common_demographic_text_Disagreement_reg)), int(.8*len(SBIC_Group_common_demographic_text_Disagreement_reg))])
print(test_Group_common_demographic_text_reg['labels'].value_counts())
train_Group_common_demographic_text_reg_oversampled = oversample(train_Group_common_demographic_text_reg)

# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_Group_common_demographic_text_reg_oversampled, eval_df = validate_Group_common_demographic_text_reg)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_Group_common_demographic_text_reg)
mean_squared_error(test_Group_common_demographic_text_reg['labels'], model_outputs)

0.0    3701
0.5    1941
1.0     353
Name: labels, dtype: int64


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/33516 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/5995 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/750 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.09304953873902559}


0.09309908361764219

In [ ]:
SBIC_Group_demographic_text_Disagreement_reg = SBIC_Disagreement[['group_demographic_text', 'normalized_disagreement_rate']]
SBIC_Group_demographic_text_Disagreement_reg = SBIC_Group_demographic_text_Disagreement_reg.rename(columns={'group_demographic_text':'text' ,'normalized_disagreement_rate': 'labels'}).drop_duplicates()
SBIC_Group_demographic_text_Disagreement_reg['labels'].value_counts()

train_Group_demographic_text_reg, validate_Group_demographic_text_reg, test_Group_demographic_text_reg = np.split(SBIC_Group_demographic_text_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(SBIC_Group_demographic_text_Disagreement_reg)), int(.8*len(SBIC_Group_demographic_text_Disagreement_reg))])
print(test_Group_demographic_text_reg['labels'].value_counts())
train_Group_demographic_text_reg_oversampled = oversample(train_Group_demographic_text_reg)

# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_Group_demographic_text_reg, eval_df = validate_Group_demographic_text_reg)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_Group_demographic_text_reg)
mean_squared_error(test_Group_demographic_text_reg['labels'], model_outputs)

0.0    3701
0.5    1941
1.0     353
Name: labels, dtype: int64


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/17983 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/5995 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/750 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.09114119526743888}


0.09115865806970525

In [ ]:
SBIC_Group_Disagreement = SBIC_Disagreement[['text', 'group_disagreement']]
SBIC_Group_Disagreement = SBIC_Group_Disagreement.rename(columns={ 'group_disagreement': 'labels'}).drop_duplicates()
SBIC_Group_Disagreement['labels'].value_counts()

0    18642
1    11331
Name: labels, dtype: int64

In [ ]:
train_reg, validate_reg, test_reg = np.split(SBIC_Group_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(SBIC_Group_Disagreement_reg)), int(.8*len(SBIC_Group_Disagreement_reg))])
train_reg['labels'].value_counts()

0.0    11172
0.5     5848
1.0      963
Name: labels, dtype: int64

In [ ]:
test_reg['labels'].value_counts()

0.0    3701
0.5    1941
1.0     353
Name: labels, dtype: int64

Over sample training set to have balanced samples

In [ ]:
train_reg_oversampled = oversample(train_reg)
train_reg_oversampled['labels'].value_counts()

0.5    11172
1.0    11172
0.0    11172
Name: labels, dtype: int64

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
# Train the model
model.train_model(train_reg_oversampled, eval_df = validate_reg)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_reg)
mean_squared_error(test_reg['labels'], model_outputs)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/33516 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4190 [00:00<?, ?it/s]

...

  0%|          | 0/5995 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/750 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.08869039026896158}


0.08874307531639718

SBIC Regression loss 0.088

In [ ]:
model_outputs

array([0.09942627, 0.02206421, 0.05847168, ..., 0.39794922, 0.15869141,
       0.43505859])

In [ ]:
wrong_predictions = pd.DataFrame(wrong_predictions)
wrong_predictions.to_csv('/content/drive/MyDrive/Morality   Legality/SBIC_predictions.csv')

SBIC classification

In [ ]:
train_args = {"num_train_epochs": 2, # optimal = 2
              "train_batch_size": 32, 
              "eval_batch_size": 32,  
              "num_labels": 3,
              "overwrite_output_dir": True}
# Create a TransformerModel
model = ClassificationModel('roberta', 'roberta-base',args=train_args)
# Train the model
model.train_model(train_reg_oversampled, eval_df = validate_reg, f1 = sklearn.metrics.f1_score)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1048 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1048 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(2096, 0.21739880504332218)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_reg, f1 = sklearn.metrics.f1_score)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5995 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/188 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.06227808498208573, 'tp': 25, 'tn': 5493, 'fp': 149, 'fn': 328, 'auroc': 0.5625945835212033, 'auprc': 0.0795858353473831, 'f1': 0.09487666034155597, 'eval_loss': 0.5954698749027305}


In [ ]:
wrong_predictions

if the disagreement is between not offensive, neutral. Then it pobably not much difference? <br>
Because the disagreement occurence is related social common sense,the disagreement dosesn't only come from text meaning but related with how the annotators recieve information, their personnal judgment and their background   
let modal train on original label first? 

# **Dilemmas**

In [ ]:
train_dilemmas_disagreement = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/train_dilemmas_disagreement.csv')
dev_dilemmas_disagreement = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/dev_dilemmas_disagreement.csv')
test_dilemmas_disagreement = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/test_dilemmas_disagreement.csv')

In [ ]:
train_dilemmas_disagreement['normalized_disagreement_rate'].value_counts()

0.5    8320
1.0    8013
0.0    7263
Name: normalized_disagreement_rate, dtype: int64

In [ ]:
test_dilemmas_disagreement['normalized_disagreement_rate'].value_counts()

0.5    808
0.0    780
1.0    772
Name: normalized_disagreement_rate, dtype: int64

labels are almost balanced. It doesn't influence prediction. <br>
loss = 0.164


In [ ]:
train_dilemmas_disagreement = train_dilemmas_disagreement[['text', 'normalized_disagreement_rate']]
dev_dilemmas_disagreement = dev_dilemmas_disagreement[['text', 'normalized_disagreement_rate']]
test_dilemmas_disagreement = test_dilemmas_disagreement[['text', 'normalized_disagreement_rate']]
train_dilemmas_disagreement= train_dilemmas_disagreement.rename(columns={'normalized_disagreement_rate': 'labels'})
dev_dilemmas_disagreement = dev_dilemmas_disagreement.rename(columns={'normalized_disagreement_rate': 'labels'})
test_dilemmas_disagreement = test_dilemmas_disagreement.rename(columns={'normalized_disagreement_rate': 'labels'})
train_dilemmas_disagreement_oversampled = oversample(train_dilemmas_disagreement)
train_dilemmas_disagreement_oversampled['labels'].value_counts()

1.0    8320
0.0    8320
0.5    8320
Name: labels, dtype: int64

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_dilemmas_disagreement_oversampled, eval_df = dev_dilemmas_disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_dilemmas_disagreement)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/24960 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/3120 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2360 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/295 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.16434755618289365}


In [ ]:
mean_squared_error(test_dilemmas_disagreement['labels'], model_outputs)

0.16434755618289365

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_dilemmas_disagreement, eval_df = dev_dilemmas_disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_dilemmas_disagreement)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/23596 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2950 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2360 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/295 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.16446041838597444}


# Social Chemistry - anticipated agreement
test prediction loss = 0.097

In [ ]:
social_agreement_disagreement = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/social_agreement_disagreement.csv')

In [ ]:
social_agreement_disagreement_reg = social_agreement_disagreement[['text', 'normalized_disagreement_rate']]
social_agreement_disagreement_reg = social_agreement_disagreement_reg.rename(columns={ 'normalized_disagreement_rate': 'labels'}).drop_duplicates()
social_agreement_disagreement_reg['labels'].value_counts()

0.000000    4353
0.333333    1546
0.666667    1165
1.000000     130
Name: labels, dtype: int64

In [ ]:
train_101_reg, validate_101_reg, test_101_reg = np.split(social_agreement_disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(social_agreement_disagreement_reg)), int(.8*len(social_agreement_disagreement_reg))])
train_101_reg['labels'].value_counts()

0.000000    2605
0.333333     921
0.666667     713
1.000000      77
Name: labels, dtype: int64

In [ ]:
test_101_reg['labels'].value_counts()

0.000000    855
0.333333    327
0.666667    232
1.000000     25
Name: labels, dtype: int64

In [ ]:
train_101_reg_oversampled = oversample(train_101_reg)
train_101_reg_oversampled['labels'].value_counts()

0.333333    2605
0.666667    2605
1.000000    2605
0.000000    2605
Name: labels, dtype: int64

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_101_reg_oversampled, eval_df = validate_101_reg)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_101_reg)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/10420 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1303 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1439 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.0763219574569828}


In [ ]:
mean_squared_error(test_101_reg['labels'], model_outputs)

0.07631569759716443

In [ ]:
# most common dempgraphics + text 
# 30-39, white, bachelor, women, 
social_agreement_disagreement['text_group_common_demographic'] = 'age: 30-39, education: bachelor, race: white, gender: women. text: ' + social_agreement_disagreement['text']

In [ ]:
#input: most common dempgraphics + text 

socialchem_group_common_demographic_text_Disagreement_reg = social_agreement_disagreement[['text_group_common_demographic','normalized_disagreement_rate']]
socialchem_group_common_demographic_text_Disagreement_reg = socialchem_group_common_demographic_text_Disagreement_reg.rename(columns={'text_group_common_demographic':'text','normalized_disagreement_rate': 'labels'}).drop_duplicates()
train_socialchem_group_common_demographic_text_Disagreement, validate_socialchem_group_common_demographic_text_Disagreement, test_socialchem_group_common_demographic_text_Disagreement = np.split(socialchem_group_common_demographic_text_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(socialchem_group_common_demographic_text_Disagreement_reg)), int(.8*len(socialchem_group_common_demographic_text_Disagreement_reg))])
train_socialchem_group_common_demographic_text_Disagreement_oversampled = oversample(train_socialchem_group_common_demographic_text_Disagreement)
print(test_socialchem_group_common_demographic_text_Disagreement['labels'].value_counts())
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_socialchem_group_common_demographic_text_Disagreement_oversampled, eval_df = validate_socialchem_group_common_demographic_text_Disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_socialchem_group_common_demographic_text_Disagreement)
mean_squared_error(test_socialchem_group_common_demographic_text_Disagreement['labels'], model_outputs)


0.000000    855
0.333333    327
0.666667    232
1.000000     25
Name: labels, dtype: int64


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/10420 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1439 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.16213556612945265}


0.1620763679073912

In [ ]:
social_agreement_disagreement['demographics'] = [" a " + str(social_agreement_disagreement['age'].iloc[i]) + ' years old ' + str(social_agreement_disagreement['school'].iloc[i]) + ' ' + str(social_agreement_disagreement['race'].iloc[i]) + ' ' + str(social_agreement_disagreement['gender'].iloc[i])  for i in range(len(social_agreement_disagreement))]
social_agreement_disagreement['group_demographic'] = social_agreement_disagreement.groupby(['text'])['demographics'].transform(lambda x:'The annotators are' + ', and'.join(x) + '. ')
social_agreement_disagreement['text_group_demographic'] = social_agreement_disagreement['group_demographic'] + social_agreement_disagreement['text']

In [ ]:
socialchem_group_demographic_text_Disagreement_reg['text'].iloc[0]

"The annotators are a 21-29 years old hs white female, and a 30-39 years old bachelor white male, and a 21-29 years old grad white male, and a 40-49 years old bachelor white female, and a 21-29 years old associate white female. It's rude to exclude people."

In [ ]:
socialchem_group_demographic_text_Disagreement_reg = social_agreement_disagreement[['text_group_demographic','normalized_disagreement_rate']]
socialchem_group_demographic_text_Disagreement_reg = socialchem_group_demographic_text_Disagreement_reg.rename(columns={'text_group_demographic':'text','normalized_disagreement_rate': 'labels'}).drop_duplicates()
train_socialchem_group_demographic_text_Disagreement, validate_socialchem_group_demographic_text_Disagreement, test_socialchem_group_demographic_text_Disagreement = np.split(socialchem_group_demographic_text_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(socialchem_group_demographic_text_Disagreement_reg)), int(.8*len(socialchem_group_demographic_text_Disagreement_reg))])
train_socialchem_group_demographic_text_Disagreement_oversampled = oversample(train_socialchem_group_demographic_text_Disagreement)
print(test_socialchem_group_demographic_text_Disagreement['labels'].value_counts())
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_socialchem_group_demographic_text_Disagreement_oversampled, eval_df = validate_socialchem_group_demographic_text_Disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_socialchem_group_demographic_text_Disagreement)
mean_squared_error(test_socialchem_group_demographic_text_Disagreement['labels'], model_outputs)


0.000000    855
0.333333    327
0.666667    232
1.000000     25
Name: labels, dtype: int64


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/10420 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1439 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.07647974168260892}


0.07647763705114784

In [ ]:
#input: demographic + text new template 
#output: group disagreement rate
social_agreement_disagreement['demographics_text_newtemp'] = ["age: " + str(social_agreement_disagreement['age'].iloc[i]) + ', education: ' + str(social_agreement_disagreement['school'].iloc[i]) + ', race: ' + str(social_agreement_disagreement['race'].iloc[i]) + ', gender: ' + str(social_agreement_disagreement['gender'].iloc[i]) + ". text: " + str(social_agreement_disagreement['text'].iloc[i]) for i in range(len(social_agreement_disagreement))]
socialchem_demo_text_newtemp_group_Disagreement_reg = social_agreement_disagreement[['demographics_text_newtemp','normalized_disagreement_rate']]
socialchem_demo_text_newtemp_group_Disagreement_reg = socialchem_demo_text_newtemp_group_Disagreement_reg.rename(columns={'demographics_text_newtemp':'text','normalized_disagreement_rate': 'labels'})
train_socialchem_demo_text_newtemp_group_Disagreement, validate_socialchem_demo_text_newtemp_group_Disagreement, test_socialchem_demo_text_newtemp_group_Disagreement = np.split(socialchem_demo_text_newtemp_group_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(socialchem_demo_text_newtemp_group_Disagreement_reg)), int(.8*len(socialchem_demo_text_newtemp_group_Disagreement_reg))])
train_socialchem_demo_text_newtemp_group_Disagreement_oversampled = oversample(train_socialchem_demo_text_newtemp_group_Disagreement)

# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_socialchem_demo_text_newtemp_group_Disagreement_oversampled, eval_df = validate_socialchem_demo_text_newtemp_group_Disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_socialchem_demo_text_newtemp_group_Disagreement)
mean_squared_error(test_socialchem_demo_text_newtemp_group_Disagreement['labels'], model_outputs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/52484 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/6561 [00:00<?, ?it/s]

...

  0%|          | 0/7194 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/900 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.015864110872466374}


0.015863774167408046

In [ ]:
#input: demographic + text
#output: group disagreement rate
social_agreement_disagreement['demographics_text'] = ["The annotator is a " + str(social_agreement_disagreement['age'].iloc[i]) + ' years old ' + str(social_agreement_disagreement['school'].iloc[i]) + ' ' + str(social_agreement_disagreement['race'].iloc[i]) + ' ' + str(social_agreement_disagreement['gender'].iloc[i]) + ". " + str(social_agreement_disagreement['text'].iloc[i]) for i in range(len(social_agreement_disagreement))]
socialchem_demo_text_group_Disagreement_reg = social_agreement_disagreement[['demographics_text','normalized_disagreement_rate']]
socialchem_demo_text_group_Disagreement_reg = socialchem_demo_text_group_Disagreement_reg.rename(columns={'demographics_text':'text','normalized_disagreement_rate': 'labels'})
train_socialchem_demo_text_group_Disagreement, validate_socialchem_demo_text_group_Disagreement, test_socialchem_demo_text_group_Disagreement = np.split(socialchem_demo_text_group_Disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(socialchem_demo_text_group_Disagreement_reg)), int(.8*len(socialchem_demo_text_group_Disagreement_reg))])
train_socialchem_demo_text_group_Disagreement_oversampled = oversample(train_socialchem_demo_text_group_Disagreement)

In [ ]:
train_socialchem_demo_text_group_Disagreement['labels'].value_counts()

0.000000    13121
0.333333     4585
0.666667     3497
1.000000      379
Name: labels, dtype: int64

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_socialchem_demo_text_group_Disagreement_oversampled, eval_df = validate_socialchem_demo_text_group_Disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_socialchem_demo_text_group_Disagreement)
mean_squared_error(test_socialchem_demo_text_group_Disagreement['labels'], model_outputs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/52484 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/6561 [00:00<?, ?it/s]

...

  0%|          | 0/7194 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/900 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.01745096914129034}


0.017449787195598407

In [ ]:
test_socialchem_demo_text_group_Disagreement['labels'].value_counts()

0.000000    4342
0.333333    1553
0.666667    1179
1.000000     120
Name: labels, dtype: int64

In [ ]:
#input: demographic + text
#output: person disagreement rate
social_agreement_disagreement['demographics_text'] = ["The annotator is a " + str(social_agreement_disagreement['age'].iloc[i]) + ' years old ' + str(social_agreement_disagreement['school'].iloc[i]) + ' ' + str(social_agreement_disagreement['race'].iloc[i]) + ' ' + str(social_agreement_disagreement['gender'].iloc[i]) + ". " + str(social_agreement_disagreement['text'].iloc[i]) for i in range(len(social_agreement_disagreement))]
socialchem_demo_text_person_Disagreement = social_agreement_disagreement[['demographics_text','person_disagreement']]
socialchem_demo_text_person_Disagreement = socialchem_demo_text_person_Disagreement.rename(columns={'demographics_text':'text','person_disagreement': 'labels'})
train_socialchem_demo_text_person_Disagreement, validate_socialchem_demo_text_person_Disagreement, test_socialchem_demo_text_person_Disagreement = np.split(socialchem_demo_text_person_Disagreement.sample(frac=1, random_state=42),[int(.6*len(socialchem_demo_text_person_Disagreement)), int(.8*len(socialchem_demo_text_person_Disagreement))])
train_socialchem_demo_text_person_Disagreement_oversampled = oversample(train_socialchem_demo_text_person_Disagreement)

# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_socialchem_demo_text_person_Disagreement_oversampled, eval_df = validate_socialchem_demo_text_person_Disagreement)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_socialchem_demo_text_person_Disagreement)
mean_squared_error(test_socialchem_demo_text_person_Disagreement['labels'], model_outputs)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/38090 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4762 [00:00<?, ?it/s]

...

  0%|          | 0/7194 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/900 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.2814412434895833}


0.28143361451926086

In [ ]:
test_socialchem_demo_text_person_Disagreement['labels'].value_counts()

0    6351
1     843
Name: labels, dtype: int64

# DynaSent
test prediction 0.1

In [ ]:
dynasent_disagreement = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/dynasent_disagreement.csv')

In [ ]:
dynasent_disagreement_reg = dynasent_disagreement[['sentence', 'normalized_disagreement_rate']]
dynasent_disagreement_reg = dynasent_disagreement_reg.rename(columns={ 'sentence': 'text','normalized_disagreement_rate': 'labels'}).drop_duplicates()
dynasent_disagreement_reg['labels'].value_counts()

0.666667    36795
0.333333    34833
0.000000    19960
1.000000    10071
Name: labels, dtype: int64

In [ ]:
train_dynasent_reg, validate_dynasent_reg, test_dynasent_reg = np.split(dynasent_disagreement_reg.sample(frac=1, random_state=42),[int(.6*len(dynasent_disagreement_reg)), int(.8*len(dynasent_disagreement_reg))])
train_dynasent_reg['labels'].value_counts()

0.666667    21940
0.333333    20992
0.000000    12095
1.000000     5968
Name: labels, dtype: int64

In [ ]:
test_dynasent_reg['labels'].value_counts()

0.666667    7414
0.333333    6913
0.000000    3970
1.000000    2035
Name: labels, dtype: int64

In [ ]:
train_dynasent_reg_oversampled = oversample(train_dynasent_reg)
train_dynasent_reg_oversampled['labels'].value_counts()

0.333333    21940
0.000000    21940
1.000000    21940
0.666667    21940
Name: labels, dtype: int64

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_dynasent_reg_oversampled, eval_df = validate_dynasent_reg)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_dynasent_reg)
mean_squared_error(test_dynasent_reg['labels'], model_outputs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/87760 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/10970 [00:00<?, ?it/s]

...

  0%|          | 0/20332 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/2542 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.09895886760278305}


0.09894837201503433

#Politeness

In [ ]:
utterances_binary_disagreement = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/utterances_binary_disagreement.csv')

In [ ]:
utterances_binary_disagreement = utterances_binary_disagreement[['text', 'disagreement_rate']]
utterances_binary_disagreement = utterances_binary_disagreement.rename(columns={'disagreement_rate': 'labels'})
utterances_binary_disagreement['labels'].value_counts()

0.0    1225
0.2     777
0.4     172
0.6       4
Name: labels, dtype: int64

In [ ]:
train_polite_reg, validate_polite_reg, test_polite_reg = np.split(utterances_binary_disagreement.sample(frac=1, random_state=42),[int(.6*len(utterances_binary_disagreement)), int(.8*len(utterances_binary_disagreement))])
train_polite_reg['labels'].value_counts()

0.0    723
0.2    475
0.4    105
0.6      3
Name: labels, dtype: int64

In [ ]:
test_polite_reg['labels'].value_counts()

0.0    262
0.2    144
0.4     30
Name: labels, dtype: int64

In [ ]:
train_polite_reg_oversampled = oversample(train_polite_reg)
train_polite_reg_oversampled['labels'].value_counts()

0.2    723
0.4    723
0.6    723
0.0    723
Name: labels, dtype: int64

In [ ]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)
# Train the model
model.train_model(train_polite_reg_oversampled, eval_df = validate_polite_reg)
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_polite_reg)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/2892 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/362 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/436 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/55 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.0211738499220122}


In [ ]:
mean_squared_error(test_polite_reg['labels'], model_outputs)

0.02133168548379743

In [ ]:
len(train_dynasent_reg)

60995